In [1]:
import numpy as np
from mindquantum.core.gates import X, H, Z, RX, RY, RZ, CNOT
from mindquantum.core.gates import Measure
from mindquantum.core.circuit import Circuit

## 模块化设计量子线路

In [2]:
def create_EPR_state(p: int, q: int) -> Circuit:
    """制备EPR Pair
    Args:
        p: 贝尔态作用第1个位置
        q: 贝尔态作用第2个位置
    Return:
        制备贝尔态线路
    """
    return Circuit([
        H(p),
        X(q, p)
    ])

def create_random_state(p: int) -> Circuit:
    """在1个比特上制备随机态，通过随机旋转初态制备
    Args:
        p: 随机值作用比特
    Return:
        处于随机的单量子态
    """
    # 使用RX, RY, RZ随机旋转角度作用在 |0> 上实现随机状态制备
    t1, t2, t3 = 2 * np.pi * np.random.random(size=3)
    return Circuit([
        RX(t1).on(p), # 不使用函数 on() 直接使用 RX(t1)(p) 也可
        RY(t2).on(p),
        RZ(t3).on(p)
    ])

def create_basic_module(p: int, q: int) -> Circuit:
    """CNOT,H,Measure 的组合在本位量子通信经常用到，综合为一个接口
    Args:
        p: 第1个量子位置
        q: 第2个量子位置
    Return:
        量子线路模块
    """
    return Circuit([
        CNOT(q, p),
        H(p),
        Measure(f'q{p}').on(p),
        Measure(f'q{q}').on(q)
    ])

In [3]:
def get_measure_result(ket_str: str, idx: int or list) -> int or list:
    """获取指定状态的测量结果，通过解析 get_qs(ket=True) 的返回值实现
    Args:
        ket_str: get_qs(ket=True)获取的量子态
        idx: 需要获取的量子位
    Return:
        指定量子位的测量结果，
    """
    ket_str2 = ket_str.split('\n')[0]
    if isinstance(idx, int):
        new_idx = -idx - 2
        return ket_str2[new_idx]
    if isinstance(idx, list):
        return [ket_str2[-i-2] for i in idx]
    print("Error: idx should be int or list!")
    return -1

## 论文线路复现

### 案例1

- Fig.3 量子瞬时传输线路

<img src="images/fig3.jpg" width=400>

In [4]:
def simulate_fig3():
    """复现论文 Fig3 的线路，实现 EPR 传输
    """
    cir_send = create_random_state(0)      # 初始需要传输的态，对应论文中 |y>
    cir_epr12  = create_EPR_state(1, 2)    # 制备 EPR
    cir_mod01 = create_basic_module(0, 1)  # 线路

    # 使用同一个随机数种子，避免各次测量结果不同
    seed = np.random.randint(0, 0xff)            # 随机数种子
    cir_all = cir_send + cir_epr12 + cir_mod01  # 量子线路
    # get_qs 不会改变系统状态
    ket_str = cir_all.get_qs(ket=True, seed=seed)
    # 获取q0, q1测量结果，保存在c0, c1
    c0, c1 = get_measure_result(ket_str, [0, 1])
    # 根据测量结果判断是否加入 X 或 Z 门
    if c1 == '1':
        cir_all += X(2)
    if c0 == '1':
        cir_all += Z(2)

    send_state = cir_send.get_qs(ket=True, seed=seed)
    recv_state = cir_all.get_qs(ket=True, seed=seed)

    print(f'Fig.3\ncomplete circuit:\n{cir_all}\n')
    print(f'Measure result:\nq0={c0}, q1={c1}\n')
    print(f'Send state ¦qn,...,q0⟩:\n{send_state}\n')
    print(f'Recv state ¦qn,...,q0⟩:\n{recv_state}\n')


simulate_fig3()

Fig.3
complete circuit:
q0: ──RX(0.102)────RY(2.666)────RZ(2.838)────●──────H──────M(q0)──
                                             │
q1: ──────H────────────●─────────────────────X────M(q1)───────────
                       │
q2: ───────────────────X──────────────────────────────────────────

Measure result:
q0=0, q1=0

Send state ¦qn,...,q0⟩:
(0.08477243949860924-0.2249188354701979j)¦0⟩
(0.15888531027657884+0.9575910448257648j)¦1⟩

Recv state ¦qn,...,q0⟩:
(0.08477243949860924-0.2249188354701979j)¦000⟩
(0.15888531027657884+0.9575910448257648j)¦100⟩



---

### 案例2

- Fig.5 量子中间方案线路

<img src="images/fig5.jpg" width=400>

In [5]:
def simulate_fig5():
    """复现 Fig.5 线路"""
    cir_send = create_random_state(0)     # the send state |y>
    cir_epr1 = create_EPR_state(1, 2)     # EPR Pair [A2, C1]
    cir_epr2 = create_EPR_state(3, 4)     # EPR Pair [C2, B]
    cir_mod01 = create_basic_module(0, 1)
    cir_mod23 = create_basic_module(2, 3)
    
    cir_all = cir_send + cir_epr1 + cir_epr2 + cir_mod01

    # 使用同一个随机数种子，避免各次测量结果不同
    seed = np.random.randint(0, 0xff)

    ## 第一次测量
    # get_qs 不会改变系统状态
    ket_str = cir_all.get_qs(ket=True, seed=seed)
    # 获取q0, q1测量结果，保存在c0, c1
    c0, c1 = get_measure_result(ket_str, [0, 1])
    # 根据测量结果判断是否加入 X 或 Z 门
    if c1 == '1':
        cir_all += X(2)
    if c0 == '1':
        cir_all += Z(2)

    ## 第二次测量
    cir_all += cir_mod23 # 加上第2部分测量
    # get_qs 不会改变系统状态
    ket_str = cir_all.get_qs(ket=True, seed=seed)
    # 获取q2, q3测量结果，保存在c2, c3
    c2, c3 = get_measure_result(ket_str, [2, 3])
    # 根据测量结果判断是否加入 X 或 Z 门
    if c3 == '1':
        cir_all += X(4)
    if c2 == '1':
        cir_all += Z(4)

    print(f'Fig.5\ncomplete circuit:\n{cir_all}\n')
    print(f'measure result:\nq0 = {c0}, q1 = {c1}\n')
    print(f'send state ¦qn,...,q0⟩:\n{cir_send.get_qs(ket=True, seed=seed)}\n')
    print(f'recv state ¦qn,...,q0⟩:\n{cir_all.get_qs(ket=True, seed=seed)}\n')


simulate_fig5()

Fig.5
complete circuit:
q0: ──RX(1.476)────RY(5.212)────RZ(2.025)────●──────H──────M(q0)───────────
                                             │
q1: ──────H────────────●─────────────────────X────M(q1)────────────────────
                       │
q2: ───────────────────X────────────X────────Z──────●────────H──────M(q2)──
                                                    │
q3: ──────H────────────●────────────────────────────X──────M(q3)───────────
                       │
q4: ───────────────────X────────────X──────────────────────────────────────

measure result:
q0 = 1, q1 = 1

send state ¦qn,...,q0⟩:
(-0.04553208008366766+0.7214669282630273j)¦0⟩
(-0.29093413911875327+0.6267133537760277j)¦1⟩

recv state ¦qn,...,q0⟩:
(-0.04553208008366766+0.7214669282630272j)¦01011⟩
(-0.2909341391187532+0.6267133537760277j)¦11011⟩



---

### 案例3

- Fig.6 在源节点和宿节点之间通过一个中间节点节点共享EPR对

<img src="images/fig6.jpg" width=400>

In [6]:
def simulate_fig6():
    cir_epr1 = create_EPR_state(0, 1) # EPR Pair [A, C1]
    cir_epr2 = create_EPR_state(2, 3) # EPR Pair [C2, B]
    cir_mod12 = create_basic_module(1, 2)
    cir_all = cir_epr1 + cir_epr2 + cir_mod12

    # 使用同一个随机数种子，避免各次测量结果不同
    seed = np.random.randint(0, 0xff) # 随机数种子

    # get_qs 不会改变系统状态
    ket_str = cir_all.get_qs(ket=True, seed=seed)
    # 获取q0, q1测量结果，保存在c0, c1
    c1, c2 = get_measure_result(ket_str, [1, 2])
    # 根据测量结果判断是否加入 X 或 Z 门
    if c2 == '1':
        cir_all += X(3)
    if c1 == '1':
        cir_all += Z(3)

    print(f'Fig.6\ncomplete circuit:\n{cir_all}\n')
    print(f'measure result:\nq1 = {c1}, q2 = {c2}\n')
    print(f'recv state ¦qn,...,q0⟩:\n{cir_all.get_qs(ket=True, seed=seed)}\n')

simulate_fig6()
# |0xx0> + |1xx1>

Fig.6
complete circuit:
q0: ──H────●─────────────────────────
           │
q1: ───────X────●──────H──────M(q1)──
                │
q2: ──H────●────X────M(q2)───────────
           │
q3: ───────X─────────────────────────

measure result:
q1 = 0, q2 = 0

recv state ¦qn,...,q0⟩:
√2/2¦0000⟩
√2/2¦1001⟩



### 案例4

- Fig.8 使用EPR桥接传输单个量子态

<img src="images/fig8.jpg" width=400>

In [7]:
def simulate_fig8():
    cir_send = create_random_state(0)
    cir_epr12 = create_EPR_state(1, 2)
    cir_epr34 = create_EPR_state(3, 4)
    cir_mod23 = create_basic_module(2, 3)
    cir_mod01 = create_basic_module(0, 1)

    cir_all = cir_send + cir_epr12 + cir_epr34 + cir_mod23

    # 使用同一个随机数种子，避免各次测量结果不同
    seed = np.random.randint(0, 0xff)

    ## 第一次测量
    # get_qs 不会改变系统状态
    ket_str = cir_all.get_qs(ket=True, seed=seed)
    # 获取q2, q3测量结果，保存在c2, c3
    c2, c3 = get_measure_result(ket_str, [2, 3])
    # 根据测量结果判断是否加入 X 或 Z 门
    if c3 == '1':
        cir_all += X(4)
    if c2 == '1':
        cir_all += Z(4)

    ## 第二次测量
    cir_all += cir_mod01 # 加上第2部分测量
    # get_qs 不会改变系统状态
    ket_str = cir_all.get_qs(ket=True, seed=seed)
    # 获取q0, q2测量结果，保存在c0, c1
    c0, c1 = get_measure_result(ket_str, [0, 1])
    # 根据测量结果判断是否加入 X 或 Z 门
    if c1 == '1':
        cir_all += X(4)
    if c0 == '1':
        cir_all += Z(4)

    print(f'Fig.8\ncomplete circuit:\n{cir_all}\n')
    print(f'measure result:\nq3={c3}, q2={c2}, q1={c1}, q0={c0}\n')
    print(f'send state ¦qn,...,q0⟩:\n{cir_send.get_qs(ket=True, seed=seed)}\n')
    print(f'recv state ¦qn,...,q0⟩:\n{cir_all.get_qs(ket=True, seed=seed)}\n')


simulate_fig8()

Fig.8
complete circuit:
q0: ──RX(4.185)────RY(4.812)────RZ(4.866)──────●────────H──────M(q0)──
                                               │
q1: ──────H────────────●───────────────────────X──────M(q1)───────────
                       │
q2: ───────────────────X────────────●──────────H──────M(q2)───────────
                                    │
q3: ──────H────────────●────────────X────────M(q3)────────────────────
                       │
q4: ───────────────────X────────────X──────────Z────────Z─────────────

measure result:
q3=1, q2=1, q1=0, q0=1

send state ¦qn,...,q0⟩:
(0.09804527980305622-0.6822699336681646j)¦0⟩
(-0.16412155535637818-0.7056620832864566j)¦1⟩

recv state ¦qn,...,q0⟩:
(0.0980452798030562-0.6822699336681646j)¦01101⟩
(-0.1641215553563781-0.7056620832864566j)¦11101⟩



### 案例5

- Fig.9 实现量子路由机制的量子线路

<img src="images/fig9.jpg" width=400>

In [8]:
def simulate_fig9():
    cir_send = create_random_state(0)
    cir_epr12 = create_EPR_state(1, 2)
    cir_epr34 = create_EPR_state(3, 4)
    cir_mod01 = create_basic_module(0, 1)
    cir_mod23 = create_basic_module(2, 3)

    cir_all = cir_send + cir_epr12 + cir_epr34 + cir_mod01 + cir_mod23

    # 使用同一个随机数种子，避免各次测量结果不同
    seed = np.random.randint(0, 0xff) # 随机数种子

    # get_qs 不会改变系统状态
    ket_str = cir_all.get_qs(ket=True, seed=seed)
    # 获取q0-q3测量结果，保存在c0-c3
    c0, c1, c2, c3 = get_measure_result(ket_str, [0, 1, 2, 3])
    a1, a2, c1, c2 = c0, c1, c2, c3
    # 根据测量结果判断是否加入 X 或 Z 门
    if a2 != c2: # XOR
        cir_all += X(4)
    if a1 != c1: # XOR
        cir_all += Z(4)

    print(f'Fig.9\ncomplete circuit:\n{cir_all}\n')
    print(f'Measure result:\nq3={c3}, q2={c2}, q1={c1}, q0={c0}\n')
    print(f'Send state ¦qn,...,q0⟩:\n{cir_send.get_qs(ket=True, seed=seed)}\n')
    print(f'Recv state ¦qn,...,q0⟩:\n{cir_all.get_qs(ket=True, seed=seed)}\n')


simulate_fig9()

Fig.9
complete circuit:
q0: ──RX(1.84)────RY(4.272)────RZ(5.627)──────●────────H──────M(q0)──
                                              │
q1: ─────H────────────●───────────────────────X──────M(q1)───────────
                      │
q2: ──────────────────X────────────●──────────H──────M(q2)───────────
                                   │
q3: ─────H────────────●────────────X────────M(q3)────────────────────
                      │
q4: ──────────────────X────────────Z─────────────────────────────────

Measure result:
q3=1, q2=1, q1=1, q0=0

Send state ¦qn,...,q0⟩:
(0.5234607273332144-0.5317360554524077j)¦0⟩
(-0.6215467966236228-0.23859005402281389j)¦1⟩

Recv state ¦qn,...,q0⟩:
(-0.5234607273332142+0.5317360554524074j)¦01110⟩
(0.6215467966236227+0.23859005402281383j)¦11110⟩



---

### 案例6

- Fig.13 Alice和Bob在同一个QBK范围内的量子线路

<img src="images/fig13.jpg" width=400>

In [9]:
def simulate_fig13():
    """same to fig9"""
    cir_send = create_random_state(0)
    cir_epr12 = create_EPR_state(1, 2)
    cir_epr34 = create_EPR_state(3, 4)
    cir_mod01 = create_basic_module(0, 1)
    cir_mod23 = create_basic_module(2, 3)

    cir_all = cir_send + cir_epr12 + cir_epr34 + cir_mod01 + cir_mod23

    # 使用同一个随机数种子，避免各次测量结果不同
    seed = np.random.randint(0, 0xff)

    # get_qs 不会改变系统状态
    ket_str = cir_all.get_qs(ket=True, seed=seed)
    # 获取q0-q3测量结果，保存在c0-c3
    c0, c1, c2, c3 = get_measure_result(ket_str, [0, 1, 2, 3])
    a1, a2, c1, c2 = c0, c1, c2, c3
    # 根据测量结果判断是否加入 X 或 Z 门
    if a2 != c2:  # XOR
        cir_all += X(4)
    if a1 != c1:  # XOR
        cir_all += Z(4)

    print(f'Fig.13\ncomplete circuit:\n{cir_all}\n')
    print(f'Measure result:\nq3={c3}, q2={c2}, q1={c1}, q0={c0}\n')
    print(f'Send state ¦qn,...,q0⟩:\n{cir_send.get_qs(ket=True, seed=seed)}\n')
    print(f'Recv state ¦qn,...,q0⟩:\n{cir_all.get_qs(ket=True, seed=seed)}\n')


simulate_fig13()

Fig.13
complete circuit:
q0: ──RX(3.658)────RY(4.457)────RZ(1.139)──────●────────H──────M(q0)──
                                               │
q1: ──────H────────────●───────────────────────X──────M(q1)───────────
                       │
q2: ───────────────────X────────────●──────────H──────M(q2)───────────
                                    │
q3: ──────H────────────●────────────X────────M(q3)────────────────────
                       │
q4: ───────────────────X──────────────────────────────────────────────

Measure result:
q3=0, q2=1, q1=0, q0=1

Send state ¦qn,...,q0⟩:
(0.5440831760500209+0.5600389380449433j)¦0⟩
(-0.48900554532345786+0.3888488936022858j)¦1⟩

Recv state ¦qn,...,q0⟩:
(0.5440831760500209+0.5600389380449432j)¦00101⟩
(-0.4890055453234578+0.38884889360228575j)¦10101⟩



### 案例7

- Fig.14 Alice和Bob在不同QBK域内时的量子线路

<img src="images/fig14.jpg" width=400>

In [10]:
def simulate_fig14():
    """复现 Fig.14 线路"""
    cir_send = create_random_state(0)  # the send state
    cir_epr12 = create_EPR_state(1, 2)
    cir_epr34 = create_EPR_state(3, 4)
    cir_epr56 = create_EPR_state(5, 6)

    cir_mod01 = create_basic_module(0, 1)
    cir_mod23 = create_basic_module(2, 3)
    cir_mod45 = create_basic_module(4, 5)

    cir_all = cir_send + cir_epr12 + cir_epr34 + cir_epr56 +\
                cir_mod01 + cir_mod23 + cir_mod45

    # 使用同一个随机数种子，避免各次测量结果不同
    seed = np.random.randint(0, 0xff)

    # get_qs 不会改变系统状态
    ket_str = cir_all.get_qs(ket=True, seed=seed)
    # 获取q0-q3测量结果，保存在c0-c3
    c0, c1, c2, c3, c4, c5 = get_measure_result(ket_str, [0, 1, 2, 3, 4, 5])
    # 根据测量结果判断是否加入 X 或 Z 门
    i0, i1, i2, i3, i4, i5 = int(c0), int(c1), int(c2), \
                                int(c3), int(c4), int(c5)
    if i1+i3+i5 in [1, 3]:  # XOR
        cir_all += X(6)
    if i0+i2+i4 in [1, 3]:  # XOR
        cir_all += Z(6)

    print(f'Fig.14\ncomplete circuit:\n{cir_all}\n')
    print(f'Measure result:\nq5={c5}, q4={c4}, q3={c3}, q2={c2}, q1={c1}, q0={c0}\n')
    print(f'Send state ¦qn,...,q0⟩:\n{cir_send.get_qs(ket=True, seed=seed)}\n')
    print(f'Recv state ¦qn,...,q0⟩:\n{cir_all.get_qs(ket=True, seed=seed)}\n')


simulate_fig14()

Fig.14
complete circuit:
q0: ──RX(2.044)────RY(0.348)────RZ(0.9)──────●────────H──────M(q0)──
                                             │
q1: ──────H────────────●─────────────────────X──────M(q1)───────────
                       │
q2: ───────────────────X───────────●─────────H──────M(q2)───────────
                                   │
q3: ──────H────────────●───────────X───────M(q3)────────────────────
                       │
q4: ───────────────────X───────────●─────────H──────M(q4)───────────
                                   │
q5: ──────H────────────●───────────X───────M(q5)────────────────────
                       │
q6: ───────────────────X────────────────────────────────────────────

Measure result:
q5=0, q4=0, q3=1, q2=0, q1=1, q0=0

Send state ¦qn,...,q0⟩:
(0.5268456231410298-0.09030861752018549j)¦0⟩
(0.44685268089308594-0.7173567623974801j)¦1⟩

Recv state ¦qn,...,q0⟩:
(0.52684562314103-0.09030861752018553j)¦0001010⟩
(0.44685268089308605-0.7173567623974798j)¦1001010⟩

